In [ ]:
import pandas as pd
import numpy as np

In [ ]:
path = '/Users/williamjames/Desktop/CSCI/487/ncaahoopR_data-master/'
seasons = os.listdir(path)
season_list = [filename for filename in seasons]
print(season_list)

In [ ]:
def getBoxScores():
    df = pd.DataFrame()
    path = '/Users/williamjames/Desktop/CSCI/487/ncaahoopR_data-master/'
    seasons = os.listdir(path)
    season_list = [filename for filename in seasons]
    for s in ['2018-19','2019-20','2020-21']:
        season_path = path + s + '/box_scores/'
        teams = os.listdir(season_path)
        team_list = [filename for filename in teams]
        print(s)
        for t in team_list:
            team_path = season_path + t + '/'
            games = os.listdir(team_path)
            game_list = [filename for filename in games]
            for i in game_list:
                game_path = team_path + i
                data = pd.read_csv(game_path)
                data['id'] = i[:-4] #create column with gameID for merging and grouping purposes
                data['team'] = t
                df = df.append(data)
    return df

In [ ]:
df = getBoxScores()
players = df.loc[~(df['player']=='TEAM')]
team = df.loc[(df['player']=='TEAM')]

In [ ]:
team = team.rename(columns={'PTS':'team_PTS','AST':'team_AST'})
players = pd.merge(players,team[['id','team','team_PTS','team_AST']],on=['id','team'])
players.loc[(players['position']=='H'),'position'] = 'G'
players['HHI'] = (players['PTS'] / players['team_PTS']) ** 2
players['HHI_assists'] = (players['AST'] / players['team_AST']) ** 2
players['G_PTS'] = 0
players.loc[(players['position']=='G'),'G_PTS'] = players['PTS']
players['F_PTS'] = 0
players.loc[(players['position']=='F'),'F_PTS'] = players['PTS']
players['C_PTS'] = 0
players.loc[(players['position']=='C'),'C_PTS'] = players['PTS']
players['starter_PTS'] = 0
players.loc[(players['starter']==True),'starter_PTS'] = players['PTS']
players['starter_MIN'] = 0
players.loc[(players['starter']==True),'starter_MIN'] = players['MIN']

In [ ]:
box = players.loc[~(players['player']=='TEAM')].groupby(['id','team'],as_index=False).agg({
    'MIN':'sum','FGM':'sum','FGA':'sum','3PTM':'sum','3PTA':'sum','FTM':'sum','FTA':'sum','OREB':'sum',
    'DREB':'sum','REB':'sum','AST':'sum','STL':'sum','BLK':'sum','TO':'sum','PF':'sum','PTS':'sum','HHI':'sum','HHI_assists':'sum','G_PTS':'sum','F_PTS':'sum','C_PTS':'sum','starter_PTS':'sum','starter_MIN':'sum','location':'max'
})

box['G_PTS'] = box['G_PTS']/box['PTS']
box['F_PTS'] = box['F_PTS']/box['PTS']
box['C_PTS'] = box['C_PTS']/box['PTS']
box['starter_PTS'] = box['starter_PTS']/box['PTS']
box['starter_MIN'] = box['starter_MIN']/box['MIN']

box['TO_rate'] = box['TO'] / (box['FGA'] + .44*box['FTA'] + box['TO'])
box['EFG'] = (box['FGM'] + .5*box['3PTM'])/box['FGA']
box['FT_rate'] = box['FTM']/box['FGA']
box['play_percent'] = box['FGM'] / (box['FGA']-box['OREB'] + box['TO'])

teams1 = box[['team','id','OREB','DREB','TO_rate','EFG','FT_rate','play_percent','PTS']][0::2]
teams2 = box[['team','id','OREB','DREB','TO_rate','EFG','FT_rate','play_percent','PTS']][1::2]

games1 = pd.merge(teams1,teams2,on=['id'])

teams11 = box[['team','id','OREB','DREB','TO_rate','EFG','FT_rate','play_percent','PTS']][1::2]
teams22 = box[['team','id','OREB','DREB','TO_rate','EFG','FT_rate','play_percent','PTS']][0::2]

games2 = pd.merge(teams11,teams22,on=['id'])

games = pd.concat([games1,games2])

games
games['OREB_rate'] = games['OREB_x'] / (games['OREB_x'] + games['DREB_y'])
games['DREB_rate'] = games['DREB_x'] / (games['DREB_x'] + games['OREB_y'])
games = games.rename(columns={'TO_rate_y':'opp_TO_rate','EFG_y':'opp_EFG','FT_rate_y':'opp_FT_rate','play_percent_y':'opp_play_percent'})
games['opp_DREB_rate'] = games['DREB_y'] / (games['DREB_y'] + games['OREB_x'])
games['opp_OREB_rate'] = games['OREB_y'] / (games['OREB_y'] + games['DREB_x'])
games['win'] = 0
games.loc[(games['PTS_x']>games['PTS_y']),'win'] = 1
games = games.rename(columns={'team_x':'team'})
box = pd.merge(box,games[['team','id','OREB_rate','DREB_rate','opp_TO_rate','opp_EFG','opp_FT_rate','opp_DREB_rate','opp_OREB_rate','opp_play_percent','win']],on=['team','id'])
box.loc[(box['location']=='H'),'location'] = 2
box.loc[(box['location']=='A'),'location'] = 0
box.loc[(box['location']=='N'),'location'] = 1

In [ ]:
stats = ['HHI','HHI_assists', 'G_PTS', 'F_PTS', 'C_PTS', 'starter_PTS','starter_MIN','TO_rate',
 'EFG', 'FT_rate','play_percent', 'OREB_rate', 'DREB_rate', 'opp_TO_rate','opp_EFG', 'opp_FT_rate', 
'opp_DREB_rate', 'opp_OREB_rate','opp_play_percent']
def add_rolling(data,stats):

    main = pd.DataFrame()
    for t in data.team.unique():
        df = data.loc[(data['team']==t)]
        for s in stats:  
            df[s] = df[s].rolling(5).mean().shift(1)
        
        main = main.append(df)
    return main

df = add_rolling(box,stats)

In [ ]:
for col in ['HHI','HHI_assists', 'G_PTS', 'F_PTS', 'C_PTS', 'starter_PTS', 'starter_MIN',
       'TO_rate', 'EFG', 'FT_rate', 'OREB_rate', 'DREB_rate', 'opp_TO_rate',
       'opp_EFG', 'opp_FT_rate', 'opp_DREB_rate','play_percent','opp_play_percent', 'opp_OREB_rate']:
       df[col] = (df[col] - df[col].mean())/df[col].std(ddof=0)

In [ ]:
df = df[['HHI','HHI_assists', 'G_PTS', 'F_PTS', 'C_PTS','location', 'starter_PTS','starter_MIN',
       'TO_rate', 'EFG', 'FT_rate', 'OREB_rate', 'DREB_rate', 'opp_TO_rate',
       'opp_EFG', 'opp_FT_rate', 'opp_DREB_rate','play_percent','opp_play_percent', 'opp_OREB_rate', 'win']]

In [ ]:
df_clean = df.dropna()
df_clean.to_csv('final_data.csv')